**Loading an image from kaggle directly in Google Colab**

In [16]:
!pip install -q kaggle
!pip install opendataset
import os
os.makedirs("/root/.kaggle", exist_ok=True)
os.environ["KAGGLE_CONFIG_DIR"] = "/root/.kaggle"
!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip

ERROR: Could not find a version that satisfies the requirement opendataset (from versions: none)
ERROR: No matching distribution found for opendataset
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other


**VGG-16 Implementation**

**Import necessary Libraries**

In [17]:
#  import tensorflow for building and training the model.
# import ImageDataGenerator for data augmentation and preprocessing.
# import VGG16 to load the pre-trained VGG-16 model.
# import Dense and Flatten layers for building the custom classification head.
# import Model to create the final model.#

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

**Data preprocessing and augmentation**

In [19]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    'chest_xray/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'chest_xray/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


 Create two ImageDataGenerator instances, one for training data with augmentation and one for validation and testing data without augmentation.

 Use flow_from_directory to load images from the respective directories and generate batches of data for training, validation, and testing.

Specify target_size to resize images to (224, 224) which is the input size for VGG-16.

 Set class_mode to 'binary' as this is a binary classification problem (pneumonia or normal).

**4. Build and compile the model:**

In [20]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


 Load the pre-trained VGG-16 model with
weights from ImageNet and exclude the top classification layers (include_top=False).

Add a custom classification head on top of the base model using Flatten, Dense, and activation functions.

Freeze the layers of the base model to prevent them from being updated during training (layer.trainable = False).

Compile the model using the Adam optimizer, binary cross-entropy loss function, and accuracy as the evaluation metric.

**5. Train the model:**

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3324s 20s/step - accuracy: 0.8472 - loss: 0.6073 - val_accuracy: 0.7500 - val_loss: 0.3326
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3270s 20s/step - accuracy: 0.9612 - loss: 0.1073 - val_accuracy: 0.6875 - val_loss: 0.6635
Epoch 3/10
 79/163 ━━━━━━━━━━━━━━━━━━━━ 27:51 20s/step - accuracy: 0.9567 - loss: 0.0995

Train the model using the fit method, providing the training and validation data generators.

Specify the number of epochs and steps per epoch.

To see the output, run the code.

**6. Evaluate the model:**

In [ ]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

****